# Maximum Gift Exchange Chaos

A model designed to explore the impact of different gift exchange rules (e.g. White Elephant, Green Grinch, etc.). The primary goal is to find the rules that lead to maximum chaos, as defined as maximum number of gift steals (possibly -- probably? -- within some time constraint).

## Gift Exchange Model

We can model a gift exchange as a game with an equal number of `G` `gifts` and `P` `players`. Players `value` each gift according to their personal preferences.

    
### Rule Variants
Some gift exchange variants use slightly different rules. These variants can be mixed and matched.

#### Additional end-game swapping
This variant adds extra swaps at the end of the game after `p1` makes their swap. If `p1` does swap with another player's gift, that player becomes active and repeats step 4, and so on until the active player elects not to swap or there are no more eligible gifts.

#### All gifts start unwrapped
In this variant, all gifts in the pool are unwrapped. This means that, instead of players learning their preferences as gifts are opened, all players have complete information about their gift preferences throughout the entire game.

#### Per-round stealing constraints
- A gift can only be stolen `St` times per round, regardless of who is doing the stealing. In other words, once a gift is stolen within a given round, it is ineligble for the remainder of that round.
- No more than `Sg` gifts can be stolen per round. Once `Sg` is reached, the round concludes and a new round begins by the next player becoming active in step 2.

#### Per-player stealing constraints
In this variant, there is also a cap on the number of times `Sp` each *player* can be stolen from. For example, once a given player `Sp_n` has been stolen from `Sp_n = Sp` times, the gift they hold at that point is now ineligible. 

#### No gift stealing constraint
In this variant, there is no cap on the number of times a given gift can be stolen. In other words, `S = infinity`.

### Gift Values
Players have personal preferences and value gifts accordingly. Each `player` has a (potentially) unique subjective `subj_value` for each `gift`. We can therefore model gift subjective value as a `G` by `P` matrix of values `subj_value_gp in range(100)`.

But, if we allow for the notion that some gifts are objectively better than others (almost all gifts will be valued higher than a spray-painted tamborine, for example), we can model `subj_value` as normally distributed variance around a common `obj_value` level. 

In [138]:
import networkx as nx
import numpy as np
import math
import pandas as pd
import scipy.stats as stats

## Inititial Conditions and Setup

### First, we set some initial conditions for our model

In [151]:
# game setup
player_count = 10
gift_count = player_count
gift_value_scale = (0,1)
gift_obj_value_distr = {'mean': .5, 'stdev': .2}
gift_subj_value_stdev = .2

# game rules
gift_steals_max = 3
player_steals_max = None
single_swap = True

# player strategies
steal_threshold = .67

### Next, we create a graph object to represent our model's state
It holds `player_count` players and an equivalent number of gifts (`gift_count`). 

Each **gift** has the following attributes:
- `type` = gift
- `open`: a boolean marking whether the gift is open (`True`) or wrapped (`False`)
- `owned`: a boolean marking whether the gift is owned or unowned; equivalent to `open` except in games where all gifts begin unwrapped
- `steals`: a counter for the number of times the gift is stolen (including swaps)
- `obj_value`: the gift's objective value, set randomly within the `gift_value_scale` based on the `gift_obj_value_distr` parameter
- `stealable`: a boolean marking whether the gift can still be stolen

Gifts are brought by a single player, which is modeled as a `brought` edge between the player and the gift. Similarly, gifts are owned by a single player and modeled as an `owns` edge between the player and their gift.

Each **player** has the following attributes:
- `type` = player
- `steals`: a counter for the number of times the player has a gift *stolen from them*
- `stealable`: a boolean marking whether the player can still be stolen *from*
- `active`: a boolean marking whether it's this player's turn to open, steal, or swap a gift

Players' subjective valuations (`subj_value`) of each gift are modeled as edges from the player to the gift. Subjective value for a given player&rightarrow;gift is set randomly around the gift's `obj_value` based on the `gift_subj_value_distr` parameter.

In [49]:
# helper function to sample one value from a truncated normal distribution
def set_gift_value(scale, mean, stdev):
    lower, upper = scale[0], scale[1]
    #mean = distr['mean']
    #stdev = distr['stdev']
    return stats.truncnorm(
        (lower - mean) / stdev,
        (upper - mean) / stdev,
        loc = mean,
        scale = stdev)

# set an objective value for a gift
def set_gift_obj_value(scale, distr):
    mean, stdev = distr['mean'], distr['stdev']
    distr = set_gift_value(scale, mean, stdev)
    return math.floor(distr.rvs(1))

# set a subjective value for a gift around its objective value
def set_gift_subj_value(scale, obj_value, stdev):
    distr = set_gift_value(scale, obj_value, stdev)
    return math.floor(distr.rvs(1))

In [97]:
# initialize a directed graph (w/ support for multiple edges)
# this will hold our players and gifts
G = nx.MultiDiGraph()


# add gifts as nodes, copying ids into a list
gifts = []

for g in range(gift_count):
    gifts.append(g)
    G.add_node(g)
    G.nodes[g]['type'] = 'gift'
    G.nodes[g]['open'] = False
    G.nodes[g]['owned'] = False
    G.nodes[g]['steals'] = 0 # number of times this gift has been stolen
    G.nodes[g]['eligible'] = True # whether this gift can still be selected or stolen, i.e. if owned = False or steals < gift_steals_max
    G.nodes[g]['obj_value'] = set_gift_obj_value(gift_value_scale, 
                                                 gift_obj_value_distr)

In [96]:
print(gifts)
print(nx.get_node_attributes(G, 'open'))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{0: False, 1: False, 2: False, 3: False, 4: False, 5: False, 6: False, 7: False, 8: False, 9: False}


In [98]:
# add players as nodes, copying ids into a list
players = []

for p in range(gift_count, gift_count + player_count):
    players.append(p)
    G.add_node(p)
    G.nodes[p]['type'] = 'player'
    G.nodes[p]['steals'] = 0 # number of times this player has been stolen *from*
    G.nodes[p]['stealable'] = True # whether this player can still be stolen *from*, i.e. if steals < player_steals_max
    G.nodes[p]['active'] = False # whether its this player's turn

    
    for g in gifts:
        # add gift brought relationship as edges between player and the gift they brought
        G.add_edge(p, g, brought = players.index(p) == gifts.index(g))

        # add player p's subjective value for each gift, as edges from player to gift
        G.add_edge(p, g, subj_value = set_gift_subj_value(gift_value_scale, 
                                                          G.nodes[g]['obj_value'], 
                                                          gift_subj_value_stdev))
        
        # add gift ownership edges between player and all gifts, initialized as False
        G.add_edge(p, g, owns = False)

In [99]:
print(players)
print(nx.get_edge_attributes(G, 'brought'))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
{(10, 0, 0): True, (10, 1, 0): False, (10, 2, 0): False, (10, 3, 0): False, (10, 4, 0): False, (10, 5, 0): False, (10, 6, 0): False, (10, 7, 0): False, (10, 8, 0): False, (10, 9, 0): False, (11, 0, 0): False, (11, 1, 0): True, (11, 2, 0): False, (11, 3, 0): False, (11, 4, 0): False, (11, 5, 0): False, (11, 6, 0): False, (11, 7, 0): False, (11, 8, 0): False, (11, 9, 0): False, (12, 0, 0): False, (12, 1, 0): False, (12, 2, 0): True, (12, 3, 0): False, (12, 4, 0): False, (12, 5, 0): False, (12, 6, 0): False, (12, 7, 0): False, (12, 8, 0): False, (12, 9, 0): False, (13, 0, 0): False, (13, 1, 0): False, (13, 2, 0): False, (13, 3, 0): True, (13, 4, 0): False, (13, 5, 0): False, (13, 6, 0): False, (13, 7, 0): False, (13, 8, 0): False, (13, 9, 0): False, (14, 0, 0): False, (14, 1, 0): False, (14, 2, 0): False, (14, 3, 0): False, (14, 4, 0): True, (14, 5, 0): False, (14, 6, 0): False, (14, 7, 0): False, (14, 8, 0): False, (14, 9, 0): False, (15, 0, 0): F

## Policies

### Game Rules and Logic

#### Basic Rules
Each player brings one gift to the game and leaves with a (typically) different gift.

The game begins with all gifts wrapped, i.e. each player knows the content of only the gift they brought and none other.

A player order is determined (by some arbitrary method, e.g. random).

1. The first player `p1` selects one of the gifts from the set of unwrapped gifts and opens it.
2. The next player `p2` is now active, and has the option to either:
    1. Open a gift from the pool, leading to a repeat of step 2 with `p3` active
    2. Steal an eligible open gift, leading to a repeat of step 2 with `p1` active
3. Repeat step 2 until all gifts have been opened (i.e. there are no gifts remaining in the pool).
4. `p1` is now active again and has the option to (forcibly) swap with another player for their eligible gift. The game ends once `p1` has swapped or decided to not swap.

A new **round** begins when a new gift is opened from the pool.

##### Stealing
- Each gift can only be stolen up to `S` times, including swaps. A given gift `G_n` is eligible for stealing (or swapping) while `S_n < S`. For example, when `S = 3`, each gift can only be stolen 3 times. After that, the gift is no longer eligible and no player may steal it or swap for it; a player holding an ineligible gift will leave the game with that gift.
- Within a given round, gifts cannot be stolen back by the player who began the turn holding that gift.

    1. If they do not swap, the game ends.
    2. If they do swap with another player, step 4 repeats with that swapped player active

### Player Strategies

1. Steal second most valuable open gift
2. Steal most valuable open gift
3. Steal open gift if `subj_value` is greater than...
    1. Average of `subj_values` of the opened gifts
    2. The maximum `subj_values` of the opened gifts
    3. An arbitrary threshold, e.g. 80
4. Steal an open gift if about to become unstealable

Possible strategies (from https://github.com/BenCasselman/YankeeSwap):
1. Player steals with probability p = (number of gifts taken) / N (naive)
2. Player always steals most valuable gift available
3. Player always steals second-most-valuable gift available (if only one gift is available, player steals that one)
4. Player never steals
5. Player steals if any stealable gift has value (to them) greater than estimated underlying value of average gift
6. Player steals about-to-be unstealable gift if one available greater than estimated underlying value of avg gift
7. Same as #5 but factor in knowledge of gift player brought.
8. Ghosh-Mahdian: Player steals if best available gift has value > theta


To keep things simple at first, we'll use a single strategy for all players: 
- steal the highest `subj_value` open gift if `subj_value` >= 0.67

In [154]:
# helper functions

# get all gift nodes
def get_gift_nodes(network):
    # gifts = extract nodes with type == gift
    # return gifts
    pass

# get IDs for open gifts that are either unowned or stealable
def get_eligible_open_giftIDs():
    # find gifts that are open and eligible and not owned by stealable==False players
    # return gifts
    pass

def get_brought_gift_for(playerID):
    # find gift for which playerID --> gift brought edge == True
    # return gift
    pass

def get_owned_gift_for(playerID):
    # find gift for which playerID --> gift owns edge == True
    # return gift
    pass

def get_eligible_known_gifts_for(playerID):
    
    # brought_gift = get_brought_gift_for(player)
    
    # eligible_open_gifts = get_eligible_open_giftIDs()
    
    # if brought_gift not in eligible_open_gifts:
        # eligible_known_gifts = eligible_open_gifts.append(brought_gift) 
        
    # else: 
        # eligible_known_gifts = eligible_open_gifts
        
    # return eligible_known_gifts
    pass
    
def get_subj_gift_values(playerID, giftIDs):
    # get player subjective gift values for each gift in giftIDs
    # return dictionary with giftIDs and subjective values
    pass
    
def sorted_subj_gift_values(playerID, giftIDs): # returns a list of tuples
    # unsorted = get_subj_gift_values(playerID, giftIDs)
    # sorted = sorted((key, value) for (key, value) in unsorted.items())
    # return sorted
    pass
    
def get_unowned_gifts():
    # find all unowned giftsIDs
    pass

def strategy_steal_if_above_threshold(player, steal_threshold):
    # eligible_known_gift_values = sorted_subj_gift_values(get_eligible_known_gifts_for(player))
    
    # best_eligible_known_gift = eligible_known_gift_values[0][0]
    # best_eligible_known_gift_value = eligible_known_gift_values[0][1]
    
    # if best_eligible_known_gift is in get_unowned_gifts():
        # gift_intention = (open_gift, best_eligible_known_gift)
        
    # elif best_eligible_known_gift_value >= steal_threshold:
        # gift_intention = (steal_gift, best_eligible_known_gift)
        
    # else: #i.e. if best_eligible_known_gift is owned and under steal_threshold
        # random_gift = select an unowned gift at random
        # gift_intention = (open_gift, random_gift)
    
    pass

def strategy_swap_gifts(player):
    # swap owned gift for gift with highest subjective value
    
    # owned_gift = 
    
    # other_gifts = 
    
    # best_other_gift = sorted_subj_gift_values(player, other_gifts)[0][1]
    
    # gift_intention = (swap_gifts, best_other_gift)
    
    pass

## State Update Functions

In [146]:
# player actions

def open_gift(active_player, gift): 
    # if gift is wrapped:
    
        # change gift's open attribute to True

        # change active_player --> gift owns edge to True
        
        # change active_player active attribute to False
        
        # change players.index(active_player + 1) active attribute to True
        
    # else:
        # throw error
    
    pass


def steal_gift(active_player, gift):
    # if gift is open and stealable and owner player is stealable:
        
        # increment owner player steals attribute
        
        # if owner player steals attribute >= player_steals_max:
            # set owner player stealable attribute to False
            
        # change previous owner player --> gift owns edge to False
        
        # change active_player --> gift owns edge to True
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_max:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # change previous owner player active attribue to True
            
    # else:
        # throw error
        
    pass

def swap_gifts(active_player, target_gift, single_swap):
    # if gift is open and stealable and owner player is stealable:
    
        # increment owner player steals attribute
        
        # if owner player steals attribuet >= player_steals_max:
            # set owner player stealable attribute to False
            
        # change previous owner player --> target_gift owns edge to False
        
        # change active_player --> target_gift owns edge to True
        
        # active_player_gift = active_player's current gift
        
        # change previous owner player --> active_player_gift owns edge to True
        
        # change active_owner --> active_player_gift owns edge to False
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_max:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # if single_swap = False:
            # change previous owner player active attribute to True
            
    # else:
        # throw error
            
    pass



In [145]:
whos

Variable                Type            Data/Info
-------------------------------------------------
G                       MultiDiGraph    
df                      DataFrame             DAYSTAMP  VALUE  ye<...>n\n[358 rows x 3 columns]
g                       int             9
gift_count              int             10
gift_obj_value_distr    dict            n=2
gift_obj_value_range    tuple           n=2
gift_steals_max         int             3
gift_subj_value_stdev   int             20
gift_value_scale        tuple           n=2
gifts                   list            n=10
math                    module          <module 'math' from '/Lib<...>th.cpython-38-darwin.so'>
np                      module          <module 'numpy' from '/Li<...>kages/numpy/__init__.py'>
nx                      module          <module 'networkx' from '<...>es/networkx/__init__.py'>
open_gift               function        <function open_gift at 0x12aa92d30>
p                       int             19
pd       

In [150]:
myset = {1, 2, 3, 4, 5}
print(myset)

myset.add(5)
print(myset)

{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}


In [153]:
mylist = [(5, 27), (3, 13)]
mylist[0][1]

27